# "Training spaCy on IDT"
> "I've forgotten where I put the output model, though"

- toc: false
- branch: master
- comments: true
- categories: [spacy, idt]


In [1]:
!git clone -b dev https://github.com/UniversalDependencies/UD_Irish-IDT

Cloning into 'UD_Irish-IDT'...
remote: Enumerating objects: 1119, done.
remote: Counting objects: 100% (181/181), done.
remote: Compressing objects: 100% (83/83), done.
remote: Total 1119 (delta 131), reused 142 (delta 98), pack-reused 938
Receiving objects: 100% (1119/1119), 12.62 MiB | 20.10 MiB/s, done.
Resolving deltas: 100% (756/756), done.


In [2]:
!mkdir idt-json

In [3]:
!python -m spacy convert /content/UD_Irish-IDT/ga_idt-ud-train.conllu /content/idt-json

✔ Generated output file (4005 documents):
/content/idt-json/ga_idt-ud-train.json


In [4]:
!python -m spacy convert /content/UD_Irish-IDT/ga_idt-ud-dev.conllu /content/idt-json

✔ Generated output file (451 documents):
/content/idt-json/ga_idt-ud-dev.json


In [5]:
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.ga.300.vec.gz
!python -m spacy init-model ga /content/ga_vectors_cc --vectors-loc cc.ga.300.vec.gz


--2021-11-17 19:27:09--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.ga.300.vec.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.75.142, 104.22.74.142, 172.67.9.4, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.75.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 184422000 (176M) [binary/octet-stream]
Saving to: ‘cc.ga.300.vec.gz’

cc.ga.300.vec.gz    100%[===================>] 175.88M  31.5MB/s    in 6.0s    

2021-11-17 19:27:16 (29.2 MB/s) - ‘cc.ga.300.vec.gz’ saved [184422000/184422000]

✔ Successfully created model
316836it [00:27, 11415.57it/s]
✔ Loaded vectors from cc.ga.300.vec.gz
✔ Sucessfully compiled vocab
317041 entries, 316836 vectors


WikiANN is currently only available through [Google Drive](https://drive.google.com/drive/folders/1Q-xdT99SeaCghihGa7nRkcXGwRGUIsKN?usp=sharing)

In [6]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [7]:
!cp /gdrive/My\ Drive/ga.tar.gz .

In [8]:
!tar zxvf ga.tar.gz

README.txt
wikiann-ga.bio


In [9]:
!wget http://downloads.dbpedia.org/links/resources/wikidatadump/2017-07-07/enwiki/20170701/enwiki-20170701-interlanguage-links_wikidataorg.ttl

--2021-11-17 19:30:46--  http://downloads.dbpedia.org/links/resources/wikidatadump/2017-07-07/enwiki/20170701/enwiki-20170701-interlanguage-links_wikidataorg.ttl
Resolving downloads.dbpedia.org (downloads.dbpedia.org)... 139.18.16.66
Connecting to downloads.dbpedia.org (downloads.dbpedia.org)|139.18.16.66|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1020894244 (974M) [text/turtle]
Saving to: ‘enwiki-20170701-interlanguage-links_wikidataorg.ttl’

enwiki-20170701-int 100%[===================>] 973.60M  22.4MB/s    in 43s     

2021-11-17 19:31:30 (22.5 MB/s) - ‘enwiki-20170701-interlanguage-links_wikidataorg.ttl’ saved [1020894244/1020894244]



In [10]:
!cat wikiann-ga.bio | awk '(NF == 7){print $6}'|sort|uniq|while read i;do grep "/$i>" enwiki-20170701-interlanguage-links_wikidataorg.ttl >> filtered;done

In [11]:
!pip install danlp 

     |████████████████████████████████| 80 kB 3.7 MB/s 
  Created wheel for progressbar: filename=progressbar-2.5-py3-none-any.whl size=12082 sha256=46a02604461966b4d49078077fa757ae4992de9c9f0b5b2db2ea521888f4cacc
  Stored in directory: /root/.cache/pip/wheels/f0/fd/1f/3e35ed57e94cd8ced38dd46771f1f0f94f65fec548659ed855
Successfully built progressbar


In [12]:
import danlp.datasets.wiki_ann
wa = danlp.datasets.wiki_ann._convert_wikiann_to_iob('wikiann-ga.bio', 'wikiann-ga.ner')

In [14]:
!head wikiann-ga.ner

Colm _ _ B-PER
Ó _ _ I-PER
Ruairc _ _ I-PER

Seosamh _ _ B-PER
Ó _ _ I-PER
Cainín _ _ I-PER

Dónal _ _ B-PER
Ó _ _ I-PER


In [15]:
!python -m spacy convert -n 10 wikiann-ga.ner /content/idt-json/

ℹ Auto-detected token-per-line NER format
ℹ Grouping every 10 sentences into a document.
✔ Generated output file (757 documents):
/content/idt-json/wikiann-ga.json


In [16]:
!rm -rf models
!mkdir models
!python -m spacy train -v /content/ga_vectors_cc -p 'tagger,parser,ner' ga models idt-json/ga_idt-ud-train.json idt-json/ga_idt-ud-dev.json

Training pipeline: ['tagger', 'parser', 'ner']
Starting with blank model 'ga'
Loading vector from model '/content/ga_vectors_cc'
Counting training words (limit=0)
/usr/lib/python3.7/runpy.py:193: UserWarning: [W022] Training a new part-of-speech tagger using a model with no lemmatization rules or data. This means that the trained model may not be able to lemmatize correctly. If this is intentional or the language you're using doesn't have lemmatization data, you can ignore this warning by setting SPACY_WARNING_IGNORE=W022. If this is surprising, make sure you have the spacy-lookups-data package installed.
  "__main__", mod_spec)

Itn  Tag Loss    Tag %    Dep Loss    UAS     LAS    NER Loss   NER P   NER R   NER F   Token %  CPU WPS
---  ---------  --------  ---------  ------  ------  ---------  ------  ------  ------  -------  -------
  1  19631.641    92.600  72419.160  80.096  66.984      0.000   0.000   0.000   0.000  100.000     8019
  2   8465.292    93.670  53795.326  82.373  72

In [41]:
!mkdir modelout

In [39]:
%cd /content
!rm -rf modelout
!rm meta.json

/content


In [40]:
%%writefile meta.json
{
  "name": "idt_sm",
  "lang": "ga",
  "version": "1.0.0",
  "spacy_version": ">=2.0.0,<3.0.0",
  "description": "Irish model for spaCy trained on IDT",
  "author": "Jim O'Regan",
  "email": "jaoregan@tcd.ie",
  "license": "CC BY-SA 3.0",
  "url": "https://huggingface.co/jimregan",
  "pipeline": ["tagger", "parser", "ner"]
}

Writing meta.json


In [42]:
!python -m spacy package --meta meta.json /content/models/model-best modelout

✔ Loaded meta.json from file
meta.json
✔ Successfully created package 'ga_idt_sm-1.0.0'
modelout/ga_idt_sm-1.0.0
To build the package, run `python setup.py sdist` in this directory.


In [25]:
!ls /content/modelout/

ga_ga_idt_sm-1.0.0  meta.json


In [43]:
%cd /content/modelout/ga_idt_sm-1.0.0
!python setup.py sdist

/content/modelout/ga_idt_sm-1.0.0
running sdist
running egg_info
creating ga_idt_sm.egg-info
writing ga_idt_sm.egg-info/PKG-INFO
writing dependency_links to ga_idt_sm.egg-info/dependency_links.txt
writing requirements to ga_idt_sm.egg-info/requires.txt
writing top-level names to ga_idt_sm.egg-info/top_level.txt
writing manifest file 'ga_idt_sm.egg-info/SOURCES.txt'
reading manifest file 'ga_idt_sm.egg-info/SOURCES.txt'
reading manifest template 'MANIFEST.in'
writing manifest file 'ga_idt_sm.egg-info/SOURCES.txt'

running check
creating ga_idt_sm-1.0.0
creating ga_idt_sm-1.0.0/ga_idt_sm
creating ga_idt_sm-1.0.0/ga_idt_sm.egg-info
creating ga_idt_sm-1.0.0/ga_idt_sm/ga_idt_sm-1.0.0
creating ga_idt_sm-1.0.0/ga_idt_sm/ga_idt_sm-1.0.0/ner
creating ga_idt_sm-1.0.0/ga_idt_sm/ga_idt_sm-1.0.0/parser
creating ga_idt_sm-1.0.0/ga_idt_sm/ga_idt_sm-1.0.0/tagger
creating ga_idt_sm-1.0.0/ga_idt_sm/ga_idt_sm-1.0.0/vocab
copying files to ga_idt_sm-1.0.0...
copying MANIFEST.in -> ga_idt_sm-1.0.0
copying m

In [44]:
!apt install git-lfs

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  git-lfs
0 upgraded, 1 newly installed, 0 to remove and 37 not upgraded.
Need to get 2,129 kB of archives.
After this operation, 7,662 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 git-lfs amd64 2.3.4-1 [2,129 kB]
Fetched 2,129 kB in 1s (1,931 kB/s)
Selecting previously unselected package git-lfs.
(Reading database ... 155219 files and directories currently installed.)
Preparing to unpack .../git-lfs_2.3.4-1_amd64.deb ...
Unpacking git-lfs (2.3.4-1) ...
Setting up git-lfs (2.3.4-1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...


In [46]:
!pip install transformers

     |████████████████████████████████| 3.1 MB 4.4 MB/s 
     |████████████████████████████████| 3.3 MB 9.8 MB/s 
     |████████████████████████████████| 59 kB 5.0 MB/s 
     |████████████████████████████████| 895 kB 17.6 MB/s 
     |████████████████████████████████| 596 kB 18.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [48]:
!transformers-cli login


WARNING! `transformers-cli login` is deprecated and will be removed in v5. Please use `huggingface-cli login` instead.

        _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
        _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
        _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
        _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
        _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

        
Username: jimregan
Password: 
Login successful
Your token: QesBBGLTWurOVdTKllaNeKyzDraDHjNlgXgiCybgXPjIwxEOJwxFtecuxzHFNpHKaLqBYjYcurRZFQxDjGsJbjDUsyYDEzUddrFbVblmFXVzoDGfueJqGXmtPELecnRr 

Your token has been saved to /root/.huggingface/token


In [54]:
!tar ztvf dist/ga_idt_sm-1.0.0.tar.gz



drwxr-xr-x root/root         0 2021-11-17 21:35 ga_idt_sm-1.0.0/
-rw-r--r-- root/root        17 2021-11-17 21:35 ga_idt_sm-1.0.0/MANIFEST.in
-rw-r--r-- root/root       242 2021-11-17 21:35 ga_idt_sm-1.0.0/PKG-INFO
drwxr-xr-x root/root         0 2021-11-17 21:35 ga_idt_sm-1.0.0/ga_idt_sm/
-rw-r--r-- root/root       291 2021-11-17 21:35 ga_idt_sm-1.0.0/ga_idt_sm/__init__.py
drwxr-xr-x root/root         0 2021-11-17 21:35 ga_idt_sm-1.0.0/ga_idt_sm/ga_idt_sm-1.0.0/
-rw-r--r-- root/root       333 2021-11-17 21:35 ga_idt_sm-1.0.0/ga_idt_sm/ga_idt_sm-1.0.0/meta.json
drwxr-xr-x root/root         0 2021-11-17 21:35 ga_idt_sm-1.0.0/ga_idt_sm/ga_idt_sm-1.0.0/ner/
-rw-r--r-- root/root       367 2021-11-17 20:59 ga_idt_sm-1.0.0/ga_idt_sm/ga_idt_sm-1.0.0/ner/cfg
-rw-r--r-- root/root   4227498 2021-11-17 20:59 ga_idt_sm-1.0.0/ga_idt_sm/ga_idt_sm-1.0.0/ner/model
-rw-r--r-- root/root        60 2021-11-17 20:59 ga_idt_sm-1.0.0/ga_idt_sm/ga_idt_sm-1.0.0/ner/moves
drwxr-xr-x root/root         0 2021-11-17

In [67]:
%cd /content/modelout/ga_idt_sm-1.0.0

/content/modelout/ga_idt_sm-1.0.0


In [68]:
!ls

dist  ga_idt_sm  ga_idt_sm.egg-info  MANIFEST.in  meta.json  setup.py


In [55]:
!transformers-cli repo create ga_idt_sm

WARNING! Managing repositories through transformers-cli is deprecated. Please use `huggingface-cli` instead.
git version 2.17.1
Error: unknown flag: --version

Sorry, no usage text found for "git-lfs"

You are about to create jimregan/ga_idt_sm
Proceed? [Y/n] Y

Your repo now lives at:
  https://huggingface.co/jimregan/ga_idt_sm

You can clone it locally with the command below, and commit/push as usual.

  git clone https://huggingface.co/jimregan/ga_idt_sm



In [69]:
!git clone https://huggingface.co/jimregan/ga_idt_sm hf

Cloning into 'hf'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 3 (delta 0), reused 0 (delta 0)
Unpacking objects: 100% (3/3), done.


In [71]:
!python -m spacy package ./ga_idt_sm ./hf # --build wheel

✔ Loaded meta.json from file
ga_idt_sm/meta.json
✔ Successfully created package 'ga_idt_sm-1.0.0'
hf/ga_idt_sm-1.0.0
To build the package, run `python setup.py sdist` in this directory.


In [72]:
!ls hf

ga_idt_sm-1.0.0


In [66]:
!ls ga_idt_sm-1.0.0/

dist  ga_idt_sm  ga_idt_sm.egg-info  MANIFEST.in  meta.json  setup.py


In [61]:
!rm -rf .git

In [ ]:
!git add .